# Libraries

In [1]:
import sklearn
from sklearn.manifold import TSNE
from sklearn.ensemble import AdaBoostClassifier
import sklearn.preprocessing
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.datasets import make_moons
import matplotlib.pyplot as plt
import warnings
from sklearn.metrics import f1_score

warnings.filterwarnings(action='ignore')

In [2]:
#raw_data 불러오기
path = './data/'
valid_raw = pd.read_csv(path+'val.csv')
train_raw = pd.read_csv(path+'train.csv')
test_raw = pd.read_csv(path+'test.csv')

#features와 target분리, ID열 제거거

valid_features = valid_raw.drop(['ID','Class'],axis=1)
valid_target = valid_raw[['Class']]

train_features = train_raw.drop('ID',axis=1)
train_ID = train_raw[['ID']]

test_features = test_raw.drop('ID',axis=1)
test_ID = test_raw[['ID']]

train_SMOTE = pd.read_csv('./SMOTE/train_smote.csv')
print(train_SMOTE.shape,train_raw.shape, valid_raw.shape, test_raw.shape)
train_SMOTE.head()

(227500, 31) (113842, 31) (28462, 32) (142503, 31)


,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V22,V23,V24,V25,V26,V27,V28,V29,V30,Class
0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,...,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,4.983721,-0.994972,0
1,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,...,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,1.418291,-0.994972,0
2,-0.425966,0.960523,1.141109,-0.168252,0.420987,-0.029728,0.476201,0.260314,-0.568671,-0.371407,...,-0.559825,-0.026398,-0.371427,-0.232794,0.105915,0.253844,0.081080,-0.256131,-0.994960,0
3,-0.644269,1.417964,1.074380,-0.492199,0.948934,0.428118,1.120631,-3.807864,0.615375,1.249376,...,-1.015455,0.057504,-0.649709,-0.415267,-0.051634,-1.206921,-1.085339,0.262698,-0.994901,0
4,-0.894286,0.286157,-0.113192,-0.271526,2.669599,3.721818,0.370145,0.851084,-0.392048,-0.410430,...,-0.268092,-0.204233,1.011592,0.373205,-0.384157,0.011747,0.142404,0.994900,-0.994901,0


In [3]:
from sklearn.ensemble import AdaBoostClassifier

# x, y 정의
X_train=train_SMOTE.drop('Class',axis=1)
y_train=train_SMOTE[['Class']]

# 모델 정의, 학습
model = AdaBoostClassifier()
model.fit(X_train,y_train)

# 예측
train_pred = model.predict(X_train)
f1_score(y_train, train_pred, average='macro')

# 모델 검증
X_val = valid_features
y_val = valid_target
val_pred = model.predict(X_val)
f1_origin=f1_score(y_val, val_pred, average='macro')
print('기본 모델 f1 score : '+ str(f1_origin))

기본 모델 f1 score : 0.8331926764657618


# best parameter set

In [4]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn import tree as st
from sklearn.metrics import classification_report

# x, y 정의
X_train=train_SMOTE.drop('Class',axis=1)
y_train=train_SMOTE[['Class']]

# 모델 정의, 학습
base_model = st.DecisionTreeClassifier(max_depth=5, random_state=0)
model = AdaBoostClassifier(base_estimator = base_model,
                           n_estimators=100, 
                           random_state=42)
model.fit(X_train,y_train)

# 예측
train_pred = model.predict(X_train)
f1_score(y_train, train_pred, average='macro')

# 모델 검증
X_val = valid_features
y_val = valid_target
val_pred = model.predict(X_val)
f1=f1_score(y_val, val_pred, average='macro')
print(f'Best F1 Score : [{f1}]')
print(classification_report(y_val,val_pred))

Best F1 Score : [0.846806907378336]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     28432
           1       0.89      0.57      0.69        30

    accuracy                           1.00     28462
   macro avg       0.95      0.78      0.85     28462
weighted avg       1.00      1.00      1.00     28462



In [ ]:
test_pred = model.predict(test_features)

In [ ]:
submit = pd.read_csv(path+'sample_submission.csv')
submit.head()
submit.shape

In [ ]:
submit['Class']=test_pred
submit.to_csv(path+'submit.csv', index=False)